# Chapter 10: Data Aggregation and Group Operations

In [1]:
import numpy as np
import pandas as pd

## 10.1 GroupBy Mechanics

**Split-apply-combine** for describing group operations:

1. Split - Data is split into groups based on one or more keys that you provide, performed an a particular axis.
2. Apply - A function is applied to each group, producing a new value.
3. Combine - Resultas of all functions are combined into a result object.

In [2]:
df = pd.DataFrame({'key1': ['a', 'a', 'b', 'b', 'a'],
                    'key2': ['one', 'two', 'one', 'two', 'one'],
                    'data1': np.random.randn(5),
                    'data2': np.random.rand(5)})
df

,key1,key2,data1,data2
0,a,one,1.256911,0.664202
1,a,two,-0.020114,0.470710
2,b,one,1.093351,0.270165
3,b,two,0.675034,0.221495
4,a,one,-0.611770,0.403440


Suppose you wanted to compute the mean of the `data1` column using the labels from `key1`.

Option 1: Access `data1` and call `groupby` with the column at `key1`.

In [3]:
grouped = df['data1'].groupby(df['key1'])
grouped

In [4]:
grouped.mean()

key1
a    0.208342
b    0.884193
Name: data1, dtype: float64

Option 2: Instead pass multiple arrays as a list to get something different.

In [5]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means

key1  key2
a     one     0.322571
      two    -0.020114
b     one     1.093351
      two     0.675034
Name: data1, dtype: float64

In [6]:
means.unstack()

key2,one,two
key1,,
a,0.322571,-0.020114
b,1.093351,0.675034


In thise example, the group keys are all Series.

In [7]:
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
years = np.array([2005, 2005, 2006, 2005, 2006])

df['data1'].groupby([states, years]).mean()

California  2005   -0.020114
            2006    1.093351
Ohio        2005    0.965973
            2006   -0.611770
Name: data1, dtype: float64

Often times the grouping information is found in the same DataFrame so just pass column names as the group keys.

In [8]:
df.groupby('key1').mean()

,data1,data2
key1,,
a,0.208342,0.512784
b,0.884193,0.245830


> Note: `df['key2']` is not numeric data, a *nuisance column*, which is excluded from the result.

In [9]:
df.groupby(['key1', 'key2']).mean()

data1     data2
key1 key2                    
a    one   0.322571  0.533821
     two  -0.020114  0.470710
b    one   1.093351  0.270165
     two   0.675034  0.221495

A generally useful GroupBy method is `size`, which returns a Series containing group sizes.

In [10]:
df.groupby(['key1', 'key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

### 10.1.1 Iterating Over Groups

The GroupBy object supports iteration, generating a sequence of 2-tuples.

In [11]:
for name, group in df.groupby('key1'):
    print(name)
    print(group)

a
  key1 key2     data1     data2
0    a  one  1.256911  0.664202
1    a  two -0.020114  0.470710
4    a  one -0.611770  0.403440
b
  key1 key2     data1     data2
2    b  one  1.093351  0.270165
3    b  two  0.675034  0.221495


In [12]:
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print((k1, k2))
    print(group)

('a', 'one')
  key1 key2     data1     data2
0    a  one  1.256911  0.664202
4    a  one -0.611770  0.403440
('a', 'two')
  key1 key2     data1    data2
1    a  two -0.020114  0.47071
('b', 'one')
  key1 key2     data1     data2
2    b  one  1.093351  0.270165
('b', 'two')
  key1 key2     data1     data2
3    b  two  0.675034  0.221495


A useful recipe is computing a dict of the data pieces as a one-liner.

In [13]:
pieces = dict(list(df.groupby('key1')))
pieces['b']

,key1,key2,data1,data2
2,b,one,1.093351,0.270165
3,b,two,0.675034,0.221495


By default, `groupby` groups on `axis=0`, but can group on any axis.

In [14]:
df.dtypes

key1      object
key2      object
data1    float64
data2    float64
dtype: object

In [15]:
grouped = df.groupby(df.dtypes, axis=1)

for dtype, group in grouped:
    print(dtype)
    print(group)

float64
      data1     data2
0  1.256911  0.664202
1 -0.020114  0.470710
2  1.093351  0.270165
3  0.675034  0.221495
4 -0.611770  0.403440
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


### 10.1.2 Selecting a Column or Subset of Columns

Indexing a GroupBy object created from a DataFrame with a column name or array of column names has the effect of column subsetting for aggregation.

    df.groupby('key1')['data1']
    df.groupby('key1')[['data2']]

is the same as:

    df['data1'].groupby(df['key1'])
    df[['data2']].groupby(df['key1'])

As an example, to compute means for just `data2` column.

In [16]:
df.groupby(['key1', 'key2'])[['data2']].mean()

data2
key1 key2          
a    one   0.533821
     two   0.470710
b    one   0.270165
     two   0.221495

In [17]:
s_grouped = df.groupby(['key1', 'key2'])['data2']
s_grouped

In [18]:
s_grouped.mean()

key1  key2
a     one     0.533821
      two     0.470710
b     one     0.270165
      two     0.221495
Name: data2, dtype: float64

### 10.1.3 Grouping with Dicts and Series

In [19]:
people = pd.DataFrame(np.random.randn(5, 5),
                        columns=['a', 'b', 'c', 'd', 'e'],
                        index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])
people.iloc[2:3, [1, 2]] = np.nan # Add a few NA values
people

,a,b,c,d,e
Joe,0.225338,1.419898,-0.676684,0.891244,-1.105182
Steve,0.504102,0.691778,-1.099005,-1.681013,0.137158
Wes,0.545056,NaN,NaN,-1.301083,0.893048
Jim,-0.703624,-0.509626,1.129142,1.176522,1.503503
Travis,-0.140369,1.225845,-0.777283,-0.785533,2.310469


Now suppose I want to sum together the columns by group.

In [20]:
mapping = {'a': 'red', 'b': 'red', 'c': 'blue',
            'd': 'blue', 'e': 'red', 'f': 'orange'} # key 'f' is unused and okay

by_column = people.groupby(mapping, axis=1)
by_column.sum()

,blue,red
Joe,0.214560,0.540054
Steve,-2.780019,1.333038
Wes,-1.301083,1.438104
Jim,2.305665,0.290252
Travis,-1.562815,3.395945


In [21]:
map_series = pd.Series(mapping)
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [22]:
people.groupby(map_series, axis=1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


### 10.1.4 Grouping with Functions

Suppose you wanted to group by length of the names; pass the `len` function.

In [23]:
people.groupby(len).sum()

,a,b,c,d,e
3,0.066769,0.910272,0.452458,0.766684,1.291368
5,0.504102,0.691778,-1.099005,-1.681013,0.137158
6,-0.140369,1.225845,-0.777283,-0.785533,2.310469


Mixing functions with arrays, dicts, or Series is okay -> converted to arrays internally.

In [24]:
key_list = ['one', 'one', 'one', 'two', 'two']
people.groupby([len, key_list]).min()

a         b         c         d         e
3 one  0.225338  1.419898 -0.676684 -1.301083 -1.105182
  two -0.703624 -0.509626  1.129142  1.176522  1.503503
5 one  0.504102  0.691778 -1.099005 -1.681013  0.137158
6 two -0.140369  1.225845 -0.777283 -0.785533  2.310469

### 10.1.5 Grouping by Index Levels

For hierarchically indexed datasets, you can aggregate using one of the levels of an axis index.

In [25]:
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'],
                                    [1, 3, 5, 1, 3]],
                                    names=['cty', 'tenor'])
hier_df = pd.DataFrame(np.random.randn(4, 5), columns=columns)
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0      1.150407  1.665293 -0.498959 -0.650646  0.171784
1     -1.864831  1.899595  0.941637  0.929387 -0.932920
2     -2.110872  1.286061 -1.815174 -0.291340  1.045797
3      1.030183  0.469391  0.379454 -1.325630 -0.010603

To group by level, pass the level number or name using `level`.

In [26]:
hier_df.groupby(level='cty', axis=1).count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


## 10.2 Data Aggregation

Aggregation refers to any data transformation that produces scalar values from arrays (`mean`, `count`, `min`, `sum`).

In [27]:
df

,key1,key2,data1,data2
0,a,one,1.256911,0.664202
1,a,two,-0.020114,0.470710
2,b,one,1.093351,0.270165
3,b,two,0.675034,0.221495
4,a,one,-0.611770,0.403440


In [28]:
grouped = df.groupby('key1')
grouped['data1'].quantile(0.9)

key1
a    1.001506
b    1.051519
Name: data1, dtype: float64

To use your own aggregation function, pass any function that aggregates an array to `aggregate` or `agg`.

In [29]:
def peak_to_peak(arr):
    return arr.max() - arr.min()

grouped.agg(peak_to_peak)

,data1,data2
key1,,
a,1.868681,0.260762
b,0.418317,0.048670


`describe` also work, even though it is not aggregations, strictly speaking.

In [30]:
grouped.describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0  0.208342  0.955058 -0.611770 -0.315942 -0.020114  0.618399   
b      2.0  0.884193  0.295795  0.675034  0.779614  0.884193  0.988772   

               data2                                                   \
           max count      mean       std       min       25%      50%   
key1                                                                    
a     1.256911   3.0  0.512784  0.135377  0.403440  0.437075  0.47071   
b     1.093351   2.0  0.245830  0.034415  0.221495  0.233663  0.24583   

                          
           75%       max  
key1                      
a     0.567456  0.664202  
b     0.257998  0.270165

> Note: Custom aggregation finctions are generally much slower than optimized functions due to extra overhead (function calls, data rearrangement).

### 10.2.1 Column-Wise and Mutiple Function Application

Let's return back to tipping dataset and add a tipping percentage column `tip_pct`.

In [31]:
tips = pd.read_csv('Examples/tips.csv')

# Add tip percentage of total bill
tips['tip_pct'] = tips['tip'] / tips['total_bill']

tips[:6]

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
5,25.29,4.71,No,Sun,Dinner,4,0.186240


Let's group the `tips` by `day` and `smoker`.

In [32]:
grouped = tips.groupby(['day', 'smoker'])

grouped_pct = grouped['tip_pct']
grouped_pct.agg('mean') # For descriptive statistics
                        # Can pass name of the function as a string

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [33]:
grouped_pct.agg(['mean', 'std', peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

If you pass a list of `(name, function)` tuples, the first element of each tuple will be a DataFrame column name (think of a list of 2-tuples as an ordered mapping).

In [34]:
grouped_pct.agg([('foo', 'mean'), ('bar', np.std)])

foo       bar
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

Suppose we wanted to compute the same three statistics for `tip_pct` and `total_bill`.

In [35]:
functions = ['count', 'mean', 'max']
result = grouped['tip_pct', 'total_bill'].agg(functions)
result

tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

In [36]:
result['tip_pct']

count      mean       max
day  smoker                           
Fri  No          4  0.151650  0.187735
     Yes        15  0.174783  0.263480
Sat  No         45  0.158048  0.291990
     Yes        42  0.147906  0.325733
Sun  No         57  0.160113  0.252672
     Yes        19  0.187250  0.710345
Thur No         45  0.160298  0.266312
     Yes        17  0.163863  0.241255

In [37]:
ftuples = [('Durchschnitt', 'mean'), ('Abweichung', np.var)]
grouped['tip_pct', 'total_bill'].agg(ftuples)

tip_pct              total_bill            
            Durchschnitt Abweichung Durchschnitt  Abweichung
day  smoker                                                 
Fri  No         0.151650   0.000791    18.420000   25.596333
     Yes        0.174783   0.002631    16.813333   82.562438
Sat  No         0.158048   0.001581    19.661778   79.908965
     Yes        0.147906   0.003767    21.276667  101.387535
Sun  No         0.160113   0.001793    20.506667   66.099980
     Yes        0.187250   0.023757    24.120000  109.046044
Thur No         0.160298   0.001503    17.113111   59.625081
     Yes        0.163863   0.001551    19.190588   69.808518

Now suppose you wanted to apply different functions to one or more of the columns. Pass a dict to `agg` that contains a mapping of column names to any of the function specifications.

In [38]:
grouped.agg({'tip': np.max, 'size': 'sum'})

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [39]:
grouped.agg({'tip_pct': ['min', 'max', 'mean', 'std'],
                'size': 'sum'})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

### 10.2.2 Returning Aggregated Data Without Row Indexes

So far, all aggregated data comes back with an index composed from the unique group key combinations. You can disable this by passing `as_index=False` to `groupby`.

In [40]:
tips.groupby(['day', 'smoker'], as_index=False).mean()

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


> Note: `reset_index` can also be used but `as_index=False` avoids unncessary computations.

## 10.3 Apply: General split-apply-combine

`apply` splits the object being manipulated into pieces, invokes the passed function on each piece, and then attempts to concatenate the pieces together.

Returning back to the tipping dataset, suppose you wanted to select the top five `tip_pct` values by group.

In [41]:
def top(df, n=5, column='tip_pct'):
    return df.sort_values(by=column)[-n:]

top(tips, n=6)

,total_bill,tip,smoker,day,time,size,tip_pct
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


In [42]:
tips.groupby('smoker').apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

The `top` function is called on each row group, then the results are glued together using `pandas.concat`, labeling the pieces with the group names.

In [43]:
tips.groupby(['smoker', 'day']).apply(top, n=1, column='total_bill')

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

In [44]:
result = tips.groupby('smoker')['tip_pct'].describe()
result

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


In [45]:
result.unstack('smoker')

       smoker
count  No        151.000000
       Yes        93.000000
mean   No          0.159328
       Yes         0.163196
std    No          0.039910
       Yes         0.085119
min    No          0.056797
       Yes         0.035638
25%    No          0.136906
       Yes         0.106771
50%    No          0.155625
       Yes         0.153846
75%    No          0.185014
       Yes         0.195059
max    No          0.291990
       Yes         0.710345
dtype: float64

When doing .groupby().describe(), it is shorthand for:

    f = lambda x: x.describe()
    grouped.apply(f)

### 10.3.1 Suppressing the Group Keys

Pass `group_keys=False` to groupby to disable hierarchical indexing with group keys.

In [46]:
tips.groupby('smoker', group_keys=False).apply(top)

,total_bill,tip,smoker,day,time,size,tip_pct
88,24.71,5.85,No,Thur,Lunch,2,0.236746
185,20.69,5.00,No,Sun,Dinner,5,0.241663
51,10.29,2.60,No,Sun,Dinner,2,0.252672
149,7.51,2.00,No,Thur,Lunch,2,0.266312
232,11.61,3.39,No,Sat,Dinner,2,0.291990
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


### 10.3.2 Quantile and Bucket Analysis

> Recall: `cut` and `qcut` are used to slice data up into buckets with bins of your choosing or by sample quantiles.

Consider a simple random dataset and an equal-length bucket categorization using `cut`.

In [47]:
frame = pd.DataFrame({'data1': np.random.rand(1000),
                        'data2': np.random.rand(1000)})
quartiles = pd.cut(frame.data1, 4)
quartiles[:10]

0          (0.75, 1.0]
1    (-0.000414, 0.25]
2          (0.25, 0.5]
3          (0.5, 0.75]
4          (0.75, 1.0]
5          (0.75, 1.0]
6          (0.25, 0.5]
7          (0.75, 1.0]
8    (-0.000414, 0.25]
9    (-0.000414, 0.25]
Name: data1, dtype: category
Categories (4, interval[float64]): [(-0.000414, 0.25] < (0.25, 0.5] < (0.5, 0.75] < (0.75, 1.0]]

In [48]:
def get_stats(group):
    return {'min': group.min(), 'max': group.max(),
            'count': group.count(), 'mean': group.mean()}

grouped = frame.data2.groupby(quartiles)
grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
"(-0.000414, 0.25]",0.005049,0.998452,262.0,0.481930
"(0.25, 0.5]",0.009305,0.995060,246.0,0.514507
"(0.5, 0.75]",0.009549,0.998232,240.0,0.506790
"(0.75, 1.0]",0.000363,0.997730,252.0,0.497242


To compute equal-size buckets based on sample quantiles, use `qcut`. Pass `labels=False` to just get quantile numbers.

In [49]:
# Return quantile numbers
grouping = pd.qcut(frame.data1, 10, labels=False)

grouped = frame.data2.groupby(grouping)
grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
0,0.013719,0.998105,100.0,0.489519
1,0.005049,0.995469,100.0,0.466032
2,0.020517,0.998452,100.0,0.515016
3,0.009305,0.995060,100.0,0.519317
4,0.017944,0.985813,100.0,0.494300
5,0.010264,0.998232,100.0,0.541576
6,0.009549,0.987284,100.0,0.508911
7,0.005484,0.986887,100.0,0.471684
8,0.001063,0.967722,100.0,0.491498


### 10.3.3 Example: Filling Missing Values with Group-Specific Values

Use `fillna` to fill in null (NA) values using a fixed value or some value derived from the data.

In [50]:
s = pd.Series(np.random.rand(6))
s[::2] = np.nan
s

0         NaN
1    0.470894
2         NaN
3    0.991827
4         NaN
5    0.065852
dtype: float64

In [51]:
s.fillna(s.mean())

0    0.509524
1    0.470894
2    0.509524
3    0.991827
4    0.509524
5    0.065852
dtype: float64

Suppose you need the fill value to vary by group. Here's sample data on US states divided into eastern and western regions.

In [52]:
states = ['Ohio', 'New York', 'Vermont', 'Florida',
            'Oregon', 'Nevada', 'California', 'Idaho']
group_key = ['East']*4 + ['West']*4
data = pd.Series(np.random.rand(8), index=states)
data

Ohio          0.619214
New York      0.853847
Vermont       0.823030
Florida       0.036446
Oregon        0.608538
Nevada        0.224962
California    0.608750
Idaho         0.633130
dtype: float64

> Note: `['East']*4` produces a list containing four copies of the elements in `['East']`. Adding lists together concatenates them.

In [53]:
data[['Vermont', 'Nevada', 'Idaho']] = np.nan
data

Ohio          0.619214
New York      0.853847
Vermont            NaN
Florida       0.036446
Oregon        0.608538
Nevada             NaN
California    0.608750
Idaho              NaN
dtype: float64

In [54]:
data.groupby(group_key).mean()

East    0.503169
West    0.608644
dtype: float64

In [55]:
fill_mean = lambda g: g.fillna(g.mean())
data.groupby(group_key).apply(fill_mean)

Ohio          0.619214
New York      0.853847
Vermont       0.503169
Florida       0.036446
Oregon        0.608538
Nevada        0.608644
California    0.608750
Idaho         0.608644
dtype: float64

In another case, you have predefined fill values in your code that vary by group. Groups have a `name` attribute set internally.

In [56]:
fill_values = {'East': 0.5, 'West': -1}
fill_func = lambda g: g.fillna(fill_values[g.name])
data.groupby(group_key).apply(fill_func)

Ohio          0.619214
New York      0.853847
Vermont       0.500000
Florida       0.036446
Oregon        0.608538
Nevada       -1.000000
California    0.608750
Idaho        -1.000000
dtype: float64

### 10.3.4 Example: Random Sampling and Permutation

Suppose you wanted to draw a random sample (with/without replacement) from a large dataset for Monte Carlo simulation. We can use `sample` method to perform the "draws".

In [57]:
# Hearts, Spades, Clubs, Diamonds
suits = ['H', 'S', 'C', 'D']
card_val = (list(range(1, 11)) + [10]*3) * 4 # Ace has value=1
base_names = ['A'] + list(range(2, 11)) + ['J', 'K', 'Q']
cards = []
for suit in ['H', 'S', 'C', 'D']:
    cards.extend(str(num) + suit for num in base_names)

deck = pd.Series(card_val, index=cards)
deck[:13]

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
dtype: int64

Drawing a hand of five cards from the deck.

In [58]:
def draw(deck, n=5):
    return deck.sample(n)

draw(deck)

6C    6
3C    3
9D    9
5D    5
7C    7
dtype: int64

Suppose you wanted two random cards from each suit. The suit is the last character of each card name (AH "Hearts", 4D "Diamonds"), we can group based on this and use `apply`.

In [59]:
get_suit = lambda card: card[-1] # last letter is suit
deck.groupby(get_suit).apply(draw, n=2)

C  8C      8
   2C      2
D  9D      9
   KD     10
H  6H      6
   3H      3
S  9S      9
   10S    10
dtype: int64

In [60]:
deck.groupby(get_suit, group_keys=False).apply(draw, n=2)

4C      4
8C      8
10D    10
JD     10
7H      7
6H      6
AS      1
JS     10
dtype: int64

### 10.3.5 Example: Group Weighted Average and Correlation

Operations between columns in a DataFrame such as group weighted average is possible. Take this dataset with group keys, values, and some weights.

In [61]:
df = pd.DataFrame({'category': ['a', 'a', 'a', 'a',
                                'b', 'b', 'b', 'b'],
                    'data': np.random.randn(8),
                    'weights': np.random.randn(8)})
df

,category,data,weights
0,a,-1.371449,-0.478800
1,a,0.144380,-0.420607
2,a,0.572747,-1.505318
3,a,0.002924,-0.334569
4,b,1.061623,-1.151700
5,b,0.218524,-1.333321
6,b,-0.287271,-0.774423
7,b,-0.590751,0.506430


In [62]:
# The group weighted average by category

grouped = df.groupby('category')
get_wavg = lambda g: np.average(g['data'], weights=g['weights'])
grouped.apply(get_wavg)

category
a    0.097552
b    0.577817
dtype: float64

Another example, consider a financial dataset obtained from Yahoo! Finance containing end-of-day prices for a few stocks and the S&P 500 index (the SPX symbol).

In [63]:
close_px = pd.read_csv('Examples/stock_px_2.csv', parse_dates=True,
                        index_col=0)
close_px.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2214 entries, 2003-01-02 to 2011-10-14
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AAPL    2214 non-null   float64
 1   MSFT    2214 non-null   float64
 2   XOM     2214 non-null   float64
 3   SPX     2214 non-null   float64
dtypes: float64(4)
memory usage: 86.5 KB


In [64]:
close_px[-4:]

,AAPL,MSFT,XOM,SPX
2011-10-11,400.29,27.00,76.27,1195.54
2011-10-12,402.19,26.96,77.16,1207.25
2011-10-13,408.43,27.18,76.37,1203.66
2011-10-14,422.00,27.27,78.11,1224.58


Suppose we want to compute a DataFrame consisting of the yearly correlations of daily returns (computed from percent changes) with SPX.

First create a function that computes the pairwise correlation of each column with the 'SPX' column.

In [65]:
spx_corr = lambda x: x.corrwith(x['SPX'])

Next compute percent change on `close_px` using `pct_change`.

In [66]:
rets = close_px.pct_change().dropna()

Group these percent changes by year.

In [67]:
get_year = lambda x: x.year
by_year = rets. groupby(get_year)
by_year.apply(spx_corr)

,AAPL,MSFT,XOM,SPX
2003,0.541124,0.745174,0.661265,1.0
2004,0.374283,0.588531,0.557742,1.0
2005,0.467540,0.562374,0.631010,1.0
2006,0.428267,0.406126,0.518514,1.0
2007,0.508118,0.658770,0.786264,1.0
2008,0.681434,0.804626,0.828303,1.0
2009,0.707103,0.654902,0.797921,1.0
2010,0.710105,0.730118,0.839057,1.0
2011,0.691931,0.800996,0.859975,1.0


In [68]:
by_year.apply(lambda g: g['AAPL'].corr(g['MSFT']))

2003    0.480868
2004    0.259024
2005    0.300093
2006    0.161735
2007    0.417738
2008    0.611901
2009    0.432738
2010    0.571946
2011    0.581987
dtype: float64

### 10.3.6 Example: Group-Wise Linear Regression

You can use `groupby` to perform more complex group-wise statistical analysis as long as the function returns a pandas object or scalar value.

Using `statsmodels` library, we'll define `regress` function which executes an ordinary least squares (OLS) regression on each chunk of data.

In [69]:
import statsmodels.api as sm

In [70]:
def regress(data, yvar, xvars):
    Y = data[yvar]
    X = data[xvars]
    X['intercept'] = 1.
    result = sm.OLS(Y, X).fit()
    return result.params

# To run a yearly linear regression of AAPL on SPX returns

by_year.apply(regress, 'AAPL', ['SPX'])

,SPX,intercept
2003,1.195406,0.000710
2004,1.363463,0.004201
2005,1.766415,0.003246
2006,1.645496,0.000080
2007,1.198761,0.003438
2008,0.968016,-0.001110
2009,0.879103,0.002954
2010,1.052608,0.001261
2011,0.806605,0.001514


## 10.4 Pivot Tables and Cross-Tabulation

A *pivot table* is a data summarization tool that aggregates a table of data by one or more keys, arranging the data in a rectangle with some of the group keys along the rows and some along the columns.

Returning to the tipping dataset, suppose you wanted to compute a table of group means (the default `pivot_table` aggregation type) arranged by `day` and `smoker` on the rows.

In [71]:
tips.pivot_table(index=['day', 'smoker'])

size       tip   tip_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

> Note: This could have been produced with `groupby` directly.

Now suppose we want to:

- Aggregate only `tip_pct` and `size` 
- Group by `time`
- Put `smoker` in the table columns
- Put `day` in the rows 

In [72]:
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'],
                    columns='smoker')

size             tip_pct          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   2.000000  2.222222  0.139622  0.165347
       Sat   2.555556  2.476190  0.158048  0.147906
       Sun   2.929825  2.578947  0.160113  0.187250
       Thur  2.000000       NaN  0.159744       NaN
Lunch  Fri   3.000000  1.833333  0.187735  0.188937
       Thur  2.500000  2.352941  0.160311  0.163863

We can include partial totals by passing `margins=True` which adds `All` row and column labels, with corresponding values being the group statistics for all data within a single tier.

In [73]:
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'],
                    columns='smoker', margins=True)

size                       tip_pct                    
smoker             No       Yes       All        No       Yes       All
time   day                                                             
Dinner Fri   2.000000  2.222222  2.166667  0.139622  0.165347  0.158916
       Sat   2.555556  2.476190  2.517241  0.158048  0.147906  0.153152
       Sun   2.929825  2.578947  2.842105  0.160113  0.187250  0.166897
       Thur  2.000000       NaN  2.000000  0.159744       NaN  0.159744
Lunch  Fri   3.000000  1.833333  2.000000  0.187735  0.188937  0.188765
       Thur  2.500000  2.352941  2.459016  0.160311  0.163863  0.161301
All          2.668874  2.408602  2.569672  0.159328  0.163196  0.160803

> Note: The `All` values are means without taking into account smoker versus non-smoker (the `All` columns) or any of the two levels of grouping on the rows (the `All` row).

To use a different aggregation function (`'count'`, `len` for count, frequency), pass it to `aggfunc`.

In [74]:
tips.pivot_table('tip_pct', index=['time', 'smoker'], columns='day',
                    aggfunc=len, margins=True)

day             Fri   Sat   Sun  Thur    All
time   smoker                               
Dinner No       3.0  45.0  57.0   1.0  106.0
       Yes      9.0  42.0  19.0   NaN   70.0
Lunch  No       1.0   NaN   NaN  44.0   45.0
       Yes      6.0   NaN   NaN  17.0   23.0
All            19.0  87.0  76.0  62.0  244.0

In [75]:
# Fill in empty (NA) values with fill_value

tips.pivot_table('tip_pct', index=['time', 'size', 'smoker'],
                    columns='day', aggfunc='mean', fill_value=0)

day                      Fri       Sat       Sun      Thur
time   size smoker                                        
Dinner 1    No      0.000000  0.137931  0.000000  0.000000
            Yes     0.000000  0.325733  0.000000  0.000000
       2    No      0.139622  0.162705  0.168859  0.159744
            Yes     0.171297  0.148668  0.207893  0.000000
       3    No      0.000000  0.154661  0.152663  0.000000
            Yes     0.000000  0.144995  0.152660  0.000000
       4    No      0.000000  0.150096  0.148143  0.000000
            Yes     0.117750  0.124515  0.193370  0.000000
       5    No      0.000000  0.000000  0.206928  0.000000
            Yes     0.000000  0.106572  0.065660  0.000000
       6    No      0.000000  0.000000  0.103799  0.000000
Lunch  1    No      0.000000  0.000000  0.000000  0.181728
            Yes     0.223776  0.000000  0.000000  0.000000
       2    No      0.000000  0.000000  0.000000  0.166005
            Yes     0.181969  0.000000  0.000000  0.158843
       3    No      0.187735  0.000000  0.000000  0.084246
            Yes     0.000000  0.000000  0.000000  0.204952
       4    No      0.000000  0.000000  0.000000  0.138919
            Yes     0.000000  0.000000  0.000000  0.155410
       5    No      0.000000  0.000000  0.000000  0.121389
       6    No      0.000000  0.000000  0.000000  0.173706

### 10.4.1 Cross-Tabulations: Crosstab

A cross-tabulation (crosstab) is a special case of a pivot table that computes group frequencies.

In [78]:
# NOTE: COPIED FROM ACCOMPANYING NOTEBOOK

from io import StringIO
data = """\
Sample  Nationality  Handedness
1   USA  Right-handed
2   Japan    Left-handed
3   USA  Right-handed
4   Japan    Right-handed
5   Japan    Left-handed
6   Japan    Right-handed
7   USA  Right-handed
8   USA  Left-handed
9   Japan    Right-handed
10  USA  Right-handed"""
data = pd.read_table(StringIO(data), sep='\s+')

In [79]:
data

,Sample,Nationality,Handedness
0,1,USA,Right-handed
1,2,Japan,Left-handed
2,3,USA,Right-handed
3,4,Japan,Right-handed
4,5,Japan,Left-handed
5,6,Japan,Right-handed
6,7,USA,Right-handed
7,8,USA,Left-handed
8,9,Japan,Right-handed
9,10,USA,Right-handed


Suppose we want to summarize by nationality and handedness. Use `pandas.crosstab`.

In [80]:
pd.crosstab(data.Nationality, data.Handedness, margins=True)

Handedness,Left-handed,Right-handed,All
Nationality,,,
Japan,2,3,5
USA,1,4,5
All,3,7,10


In [81]:
pd.crosstab([tips.time, tips.day], tips.smoker, margins=True)

smoker        No  Yes  All
time   day                
Dinner Fri     3    9   12
       Sat    45   42   87
       Sun    57   19   76
       Thur    1    0    1
Lunch  Fri     1    6    7
       Thur   44   17   61
All          151   93  244

## 10.5